# The bagholder project: Measuring stock purchase price of apes

## 1. Get price data of GME over the last 5 years

I used data from Yahoo Finance. Downloaded a CSV

In [1]:
import pandas as pd
import requests
import time
from bisect import *
import hashlib

In [2]:
stock_prices = pd.read_csv('data/GME.csv')

## 2. Scrap ape's purchase data from their own API =) 

I have no desire or financial resources to make my own scraper, so I used one that the ape made :)

In [3]:
# Scrap API and get a df of name -> (total_cost, share_count)
API_URL = 'https://5o7q0683ig.execute-api.us-west-2.amazonaws.com/prod/computershared/posts'

# This api returns max 500 records, and you gotta get the u and id of the 500th user & id to get the next batch of 500 records
def make_url(u=None, id=None):
    if u == None:
        return API_URL
    return f"{API_URL}?resumeUser={u}&resumeId={id}"
    
def get_day_str(epoch_time):
    return time.strftime('%Y-%m-%d', time.localtime(epoch_time))
    
# Will find the day nearest time posted and get stock prices from that day
# Ex: If post posted on Sunday, will get closing price on Friday
def get_stock_price(epoch_time):
    day = get_day_str(epoch_time)
    idx = bisect_right(stock_prices['Date'], day)
    if idx == 0:
        return stock_prices.iloc[0]['Close']
    return float(stock_prices.iloc[idx-1]['Close'])
    
# Return a df of name -> (total_cost, share_count)
def scrap_api_data(limit=1000):
    u,id = None,None
    purchased_data = pd.DataFrame(columns = ['User', 'Total Cost', 'Share Count'])
    while limit > 0:
        response = requests.get(make_url(u, id))
        json_data = response.json()
        count = json_data['Count']
        limit -= int(count)
        for item in json_data['Items']:
            posted_time = int(float(item['timestamp']['N'])) # For some reason the API spits out float for timestamp bruh
            day = get_day_str(posted_time)
            purchased_price = get_stock_price(posted_time)
            user = hash(item['u']['S']) % (10 ** 8) # hash user id for anonymous. though, nothing is preventing you from removing it...
            share_count = float(item['shares']['N'])
            
            if user not in purchased_data['User'].values:
                purchased_data.loc[len(purchased_data.index)] = [user, 0, 0]
            
            idx = purchased_data['User'] == user
            purchased_data.loc[idx,'Total Cost'] += share_count * purchased_price
            purchased_data.loc[idx,'Share Count'] += share_count
            
            # print(user, day, purchased_price, share_count)
            
        # Go to next 500
        if 'LastEvaluatedKey' not in json_data:
            print("FUDDDDD")
            return purchased_data
        u = json_data['LastEvaluatedKey']['u']['S']
        id = json_data['LastEvaluatedKey']['id']['S']
        print(f"Next 500, {limit} left")
    return purchased_data

In [4]:
purchased_data = scrap_api_data(limit=50000)
purchased_data

Next 500, 49500 left
Next 500, 49000 left
Next 500, 48500 left
Next 500, 48000 left
Next 500, 47500 left
Next 500, 47000 left
Next 500, 46500 left
Next 500, 46000 left
Next 500, 45500 left
Next 500, 45000 left
Next 500, 44500 left
Next 500, 44000 left
Next 500, 43500 left
Next 500, 43000 left
Next 500, 42500 left
Next 500, 42000 left
Next 500, 41500 left
Next 500, 41000 left
Next 500, 40500 left
Next 500, 40000 left
Next 500, 39500 left
Next 500, 39000 left
Next 500, 38500 left
Next 500, 38000 left
Next 500, 37500 left
Next 500, 37000 left
Next 500, 36500 left
Next 500, 36000 left
Next 500, 35500 left
Next 500, 35000 left
Next 500, 34500 left
Next 500, 34000 left
Next 500, 33500 left
Next 500, 33000 left
Next 500, 32500 left
Next 500, 32000 left
Next 500, 31500 left
Next 500, 31000 left
Next 500, 30500 left
Next 500, 30000 left
Next 500, 29500 left
Next 500, 29000 left
Next 500, 28500 left
Next 500, 28000 left
Next 500, 27500 left
Next 500, 27000 left
Next 500, 26500 left
Next 500, 260

,User,Total Cost,Share Count
0,57621502,10757.000205,409.0
1,64953664,1779.373333,58.197
2,6954785,548.212515,15.0
3,36946269,6612.181104,121.413537
4,82954505,120.6425,3.0
...,...,...,...
19166,35459067,2292.405852,62.38
19167,12347915,3201.187425,75.0
19168,1921864,10582.879015,305.136
19169,54966068,4766.689475,137.945


# Who's the biggest bagholder?

## Get user with the largest average price

In [5]:
purchased_data['Average Price'] = purchased_data['Total Cost'] / purchased_data['Share Count']
purchased_data.sort_values(by='Average Price', ascending=False)

,User,Total Cost,Share Count,Average Price
3982,88307917,185.662503,3.0,61.887501
17995,77042094,371.325006,6.0,61.887501
6109,75774816,185.662503,3.0,61.887501
12475,68207673,371.325006,6.0,61.887501
10537,86493449,1485.300024,24.0,61.887501
...,...,...,...,...
6942,15879239,1493.61942,76.488,19.5275
19109,67790993,1757.475,90.0,19.5275
6521,57188914,175.7475,9.0,19.5275
10371,68673851,68346.25,3500.0,19.5275


# Who's the biggest whale?

## Get user with the largest share count and position

In [6]:
purchased_data.sort_values(by='Total Cost', ascending=False)

,User,Total Cost,Share Count,Average Price
937,45464400,4549748.775438,128469.0,35.415149
5473,70561731,3174271.37507,70022.977,45.331854
10925,75223238,2597094.22374,66603.130851,38.993576
8152,25613174,2397131.930381,71315.584,33.613017
16207,78368099,2094660.777887,55075.278,38.032686
...,...,...,...,...
10821,49121231,4.738492,0.129,36.732498
14184,96870042,2.354033,0.09402,25.0375
1686,41521418,1.89948,0.044,43.169998
13493,99726365,0.184743,0.004203,43.955002


In [7]:
purchased_data.sort_values(by='Share Count', ascending=False)

,User,Total Cost,Share Count,Average Price
937,45464400,4549748.775438,128469.0,35.415149
8152,25613174,2397131.930381,71315.584,33.613017
5473,70561731,3174271.37507,70022.977,45.331854
10925,75223238,2597094.22374,66603.130851,38.993576
11410,79552341,1866356.755717,60435.0,30.882051
...,...,...,...,...
14184,96870042,2.354033,0.09402,25.0375
3199,67307754,4.749243,0.092977,51.080002
1686,41521418,1.89948,0.044,43.169998
13493,99726365,0.184743,0.004203,43.955002


# Price is right: Comparing GME hodler's number to current share price

In [30]:
df = purchased_data
current_price = 33.78
total_share = df['Share Count'].sum()
total_cost = df['Total Cost'].sum()
average_price = total_cost / total_share
print(f"Total share purchased: {round(total_share, 2):,}")
print(f"Total cost: {round(total_cost, 2):,}")
print(f"Average price of ape: {round(average_price, 2):,}")
print(f"Are they up? {round((current_price-average_price)/current_price * 100, 2):,}%")
print(f"Average cost: {round(df['Total Cost'].mean(), 2):,}")
print(f"Average holding: {round(df['Share Count'].mean(), 2):,}")
print(f"Median cost: {round(df['Total Cost'].median(), 2):,}")
print(f"Median holding: {round(df['Share Count'].median(), 2):,}")
print(f"Percentage of apes who are down: {round(df.loc[df['Average Price'] > current_price, 'Average Price'].count() / df['Average Price'].count() * 100, 2):,}%")

Total share purchased: 6,045,363.26
Total cost: 222,059,443.43
Average price of ape: 36.73
Are they up? -8.74%
Average cost: 11,583.09
Average holding: 315.34
Median cost: 1,829.3
Median holding: 50.0
Percentage of apes who are down: 65.35%
